**Why Use DuckDB Instead of Only pandas**

Speed on large datasets
- DuckDB is optimized for analytics on columnar data like Parquet and runs faster than pandas when working with millions of rows.

Query Parquet directly
- You do not need to load entire files into memory. DuckDB queries Parquet partitions directly on disk.

SQL interface
- You can use familiar SQL (GROUP BY, EXTRACT, JOIN) inside Python, and return results as pandas DataFrames.

Memory efficient
- DuckDB only brings back the aggregated results instead of loading all rows into RAM.

Scales easily
-The same queries work for two months of data or multiple years of data without changing code.

Complements pandas
- Use pandas for smaller data exploration and custom calculations, and DuckDB for heavy queries and large datasets.

In [ ]:
# Install DuckDB package (only needed the first time in a new environment)
!pip install duckdb

# Import required libraries
import duckdb       # DuckDB: in-process SQL engine for analytics
import sys          # sys: lets us query system and Python runtime info
import os           # os: for operating system utilities (file paths, env vars)
import pathlib      # pathlib: modern, object-oriented filesystem paths

# Print current Python version (just the first part, e.g., "3.11.9")
print("Python:", sys.version.split()[0])

# Print installed DuckDB version (e.g., "1.1.2")
print("DuckDB:", duckdb.__version__)


In [ ]:
# Define the DuckDB database file path (will create if it doesn’t exist yet)
db_path = "nyc.duckdb"       

# Connect to the DuckDB database file (creates file if missing)
con = duckdb.connect(db_path)

# Set a DuckDB runtime setting: allow queries to use up to 4 CPU threads
con.execute("PRAGMA threads=4;")  

# Return two things:
#   1. The database path string
#   2. Whether the file currently exists on disk (True/False)
db_path, os.path.exists(db_path)   


In [20]:
# Create (or replace) a DuckDB VIEW named "yellow"
# This view points to the cleaned, partitioned Parquet files we saved earlier.
# We use a glob path ('year=2024/month=*/**/*.parquet') to match all Jan+Feb parquet files.
con.execute("""
  CREATE OR REPLACE VIEW yellow AS
  SELECT *
  FROM read_parquet('data/warehouse/yellow_clean/year=2024/month=*/**/*.parquet');
""")

# Run a quick sanity check SQL query:
# - MIN(tpep_pickup_datetime): earliest pickup timestamp in the dataset
# - MAX(tpep_pickup_datetime): latest pickup timestamp
# - COUNT(*): total number of rows
# Convert the DuckDB result to a pandas DataFrame with .df()
con.sql("""
    SELECT 
        MIN(tpep_pickup_datetime) AS min_ts, 
        MAX(tpep_pickup_datetime) AS max_ts, 
        COUNT(*) AS rows 
    FROM yellow
""").df()


,min_ts,max_ts,rows
0,2024-01-01,2024-03-01 00:01:37,8719170


In [21]:
# Trips by hour --------------------------------------------
trips_by_hour = con.sql("""
  SELECT 
    EXTRACT(hour FROM tpep_pickup_datetime) AS hour,  -- extract pickup hour (0–23)
    COUNT(*) AS trips                                -- count total trips in that hour
  FROM yellow
  WHERE year=2024 AND month IN (1,2)                 -- filter Jan + Feb 2024 only
  GROUP BY 1                                         -- group by the extracted hour
  ORDER BY 1                                         -- sort ascending by hour
""").df()                                            # convert DuckDB result to pandas DataFrame

# Revenue by month -----------------------------------------
rev_by_month = con.sql("""
  SELECT 
    strftime(tpep_pickup_datetime, '%Y-%m') AS ym,   -- format pickup datetime as "YYYY-MM"
    COUNT(*) AS trips,                               -- number of trips that month
    SUM(total_amount) AS revenue,                    -- total revenue
    AVG(total_amount) AS avg_total                   -- average revenue per trip
  FROM yellow
  WHERE year=2024 AND month IN (1,2)                 -- restrict to Jan + Feb 2024
  GROUP BY 1                                         -- group by formatted year-month
  ORDER BY 1                                         -- sort chronologically by ym
""").df()

# Revenue by payment type -------------------------------
rev_by_pay = con.sql("""
  SELECT
    CASE payment_type                                -- map numeric codes to human labels
      WHEN 1 THEN 'Credit card'
      WHEN 2 THEN 'Cash'
      WHEN 3 THEN 'No charge'
      WHEN 4 THEN 'Dispute'
      WHEN 5 THEN 'Unknown'
      WHEN 6 THEN 'Voided trip'
      ELSE 'Other' END AS payment_type_name,
    COUNT(*) AS trips,                               -- number of trips by payment type
    SUM(total_amount) AS total_revenue,              -- total revenue by payment type
    AVG(total_amount) AS avg_total                   -- average trip total by payment type
  FROM yellow
  WHERE year=2024 AND month IN (1,2)                 -- limit to Jan + Feb 2024
  GROUP BY 1                                         -- group by payment_type_name
  ORDER BY total_revenue DESC                        -- rank by revenue, highest first
""").df()

# Show first few rows of each result to verify
trips_by_hour.head(), rev_by_month.head(), rev_by_pay.head()


(   hour   trips
 0     0  233755
 1     1  159178
 2     2  108748
 3     3   70245
 4     4   47452,
         ym    trips       revenue  avg_total
 0  2024-01  2880425  7.859816e+07  27.287001
 1  2024-02  5838741  1.586736e+08  27.175989,
   payment_type_name    trips  total_revenue  avg_total
 0       Credit card  6935089   1.940975e+08  27.987745
 1              Cash  1203263   2.853025e+07  23.710738
 2             Other   487106   1.235331e+07  25.360627
 3           Dispute    64417   1.633888e+06  25.364236
 4         No charge    29291   6.567632e+05  22.422014)

In [ ]:
# Imports
import duckdb         # SQL-on-Parquet engine (in-process, fast)
import pandas as pd   # DataFrames for model input/output
import os             # to check file existence

# Path to your existing DuckDB database file
db_path = "nyc.duckdb"     # change if your file has a different name

# Connect (creates the file if it doesn't exist)
con = duckdb.connect(db_path)

# Let DuckDB use multiple threads for speed (tweak as desired)
con.execute("PRAGMA threads=4;")

# Confirm the DB file exists now
print("DuckDB file:", db_path, "| exists:", os.path.exists(db_path))

In [ ]:
# Check how many missing values each column has in the modeling DataFrame
df.isna().sum()  # quick scan: look for non-zero counts

In [ ]:
# Drop rows with missing target; keep only rows where tip_amount is present
df_model = df.dropna(subset=["tip_amount"]).copy()  # create a clean copy with target present

# Define target and features
y = df_model["tip_amount"].astype(float)  # target: tip in USD
X = df_model.drop(columns=["tip_amount"]) # features: everything else

In [ ]:
# Imports for preprocessing and modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  # <-- imputer for NaNs

# Identify numeric and categorical features 
numeric_features = ["trip_distance", "duration_min", "passenger_count", "pickup_hour", "month", "dow", "is_weekend"]
categorical_features = ["payment_type", "RatecodeID"] + ([ "PULocationID", "DOLocationID" ] if "PULocationID" in X.columns else [])

# Numeric pipeline: impute missing with median
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))  # fill NaNs in numeric columns with median
])

# Categorical pipeline: impute missing with most frequent, then one-hot encode
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),       # fill NaNs in categoricals with mode
    ("ohe", OneHotEncoder(handle_unknown="ignore"))             # one-hot encode; ignore unseen labels at predict time
])

# ColumnTransformer to apply pipelines to the right columns
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),            # apply numeric pipeline to numeric columns
        ("cat", categorical_pipeline, categorical_features),    # apply categorical pipeline to categorical columns
    ],
    remainder="drop"                                            # drop any columns not listed above
)

# Train/test split (random; for time-aware use Jan=train, Feb=test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)



In [ ]:
# Models and metrics
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Define a few baseline regressors
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42),
}

# Fit/evaluate each model in a pipeline with preprocessing
rows = []  # collect results here
for name, model in models.items():
    pipe = Pipeline([("prep", preprocess), ("model", model)])  # preprocessing + estimator
    pipe.fit(X_train, y_train)                                  # train
    preds = pipe.predict(X_test)                                # predict on holdout

    rmse = mean_squared_error(y_test, preds, squared=False)     # RMSE
    mae  = mean_absolute_error(y_test, preds)                   # MAE
    r2   = r2_score(y_test, preds)                              # R²

    rows.append({"model": name, "RMSE": rmse, "MAE": mae, "R2": r2})

# Leaderboard
results = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)
print(results)


In [ ]:
# scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define target and features
y = df["tip_amount"].astype(float)     # target: tip in USD
X = df.drop(columns=["tip_amount"])    # all other columns are features

# Separate numeric and categorical columns
numeric_features = ["trip_distance", "duration_min", "passenger_count", "pickup_hour", "month", "dow", "is_weekend"]
categorical_features = ["payment_type", "RatecodeID"] + (["PULocationID", "DOLocationID"] if use_locations else [])

# Preprocess: pass numeric through, one-hot encode categoricals
preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),                         # keep numeric as-is
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_features),  # OHE for cats
    ],
    remainder="drop"                                                      # drop anything else
)

# Split data (random split; for time-aware, train=Jan, test=Feb)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)


In [ ]:
# Models and metrics
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Define a few models to compare
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42),
}

# Train/evaluate each model inside a pipeline with preprocessing
rows = []  # to collect results
for name, model in models.items():
    pipe = Pipeline([("prep", preprocess), ("model", model)])  # preprocessing + estimator
    pipe.fit(X_train, y_train)                                  # train
    preds = pipe.predict(X_test)                                # predict on holdout

    rmse = mean_squared_error(y_test, preds, squared=False)     # RMSE
    mae  = mean_absolute_error(y_test, preds)                   # MAE
    r2   = r2_score(y_test, preds)                              # R²

    rows.append({"model": name, "RMSE": rmse, "MAE": mae, "R2": r2})

# Leaderboard sorted by RMSE (lower is better)
results = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)
print(results)
